In [ ]:
# First cell - Install required packages
!pip install gradio
!pip install requests

# Second cell - Import libraries and create the translation app
import gradio as gr
import requests
import uuid
import json
from typing import List, Dict
import os
from datetime import datetime

class AzureTranslator:
    def __init__(self, key: str, location: str):
        self.key = key
        self.location = location
        self.endpoint = "https://api.cognitive.microsofttranslator.com"
        self.path = '/translate'
        self.constructed_url = self.endpoint + self.path

    def translate_text(self, text: str, from_lang: str, to_langs: List[str]) -> Dict:
        if not text.strip():
            return {"error": "Text cannot be empty"}

        params = {
            'api-version': '3.0',
            'from': from_lang,
            'to': to_langs
        }

        headers = {
            'Ocp-Apim-Subscription-Key': self.key,
            'Ocp-Apim-Subscription-Region': self.location,
            'Content-type': 'application/json',
            'X-ClientTraceId': str(uuid.uuid4())
        }

        body = [{
            'text': text
        }]

        try:
            response = requests.post(
                self.constructed_url,
                params=params,
                headers=headers,
                json=body
            )

            if response.status_code == 401:
                return {"error": "Authentication failed. Please check your API key and location."}

            response.raise_for_status()
            return response.json()

        except requests.exceptions.ConnectionError:
            return {"error": "Connection failed. Please check your internet connection."}
        except requests.exceptions.RequestException as e:
            return {"error": f"An error occurred: {str(e)}"}

def translate_interface(text: str, source_lang: str, target_langs: str, api_key: str, api_region: str) -> str:
    """Interface function for Gradio"""
    if not text.strip():
        return "Error: Please enter text to translate"

    if not api_key.strip():
        return "Error: Please enter your Azure Translator API key"

    if not api_region.strip():
        return "Error: Please enter your Azure region"

    # Convert comma-separated languages to list
    target_lang_list = [lang.strip() for lang in target_langs.split(',') if lang.strip()]

    if not target_lang_list:
        return "Error: Please enter valid target languages"

    # Initialize translator
    translator = AzureTranslator(api_key, api_region)

    # Perform translation
    results = translator.translate_text(
        text=text,
        from_lang=source_lang,
        to_langs=target_lang_list
    )

    # Check for errors
    if "error" in results:
        return f"Translation error: {results['error']}"

    # Format results
    output = "Translation Results:\n\n"
    for translation in results[0]['translations']:
        output += f"To {translation['to']}:\n{translation['text']}\n\n"

    return output

# Create the Gradio interface
def create_translation_app():
    with gr.Blocks(title="Azure Translator App") as app:
        gr.Markdown("""
        # 🌐 Azure Translator App
        Translate text to multiple languages using Azure Translator Service.

        ### Instructions:
        1. Enter your Azure Translator API key and region
        2. Enter the text you want to translate
        3. Specify source and target languages using language codes (e.g., en, es, fr)
        4. Click 'Translate' to see the results
        """)

        with gr.Row():
            with gr.Column():
                api_key = gr.Textbox(
                    label="Azure Translator API Key",
                    placeholder="Enter your API key here",
                    type="password"
                )
                api_region = gr.Textbox(
                    label="Azure Region",
                    placeholder="e.g., eastus"
                )

        with gr.Row():
            with gr.Column():
                text_input = gr.Textbox(
                    label="Text to Translate",
                    placeholder="Enter text here",
                    lines=5
                )
                source_lang = gr.Textbox(
                    label="Source Language Code",
                    placeholder="e.g., en",
                    value="en"
                )
                target_langs = gr.Textbox(
                    label="Target Language Codes (comma-separated)",
                    placeholder="e.g., es,fr,de",
                    value="es,fr"
                )
                translate_btn = gr.Button("Translate", variant="primary")

            with gr.Column():
                output = gr.Textbox(
                    label="Translation Results",
                    lines=10,
                    placeholder="Translation results will appear here..."
                )

        # Connect the interface
        translate_btn.click(
            fn=translate_interface,
            inputs=[text_input, source_lang, target_langs, api_key, api_region],
            outputs=output
        )

        gr.Markdown("""
        ### Common Language Codes:
        - English: en
        - Spanish: es
        - French: fr
        - German: de
        - Italian: it
        - Portuguese: pt
        - Chinese: zh
        - Japanese: ja
        - Korean: ko
        - Russian: ru

        [View full list of language codes](https://learn.microsoft.com/en-us/azure/ai-services/translator/language-support)
        """)

    return app

# Create and launch the app
app = create_translation_app()
app.launch(debug=True, share=True)

  Using cached semantic_version-2.10.0-py2.py3-none-any.whl.metadata (9.7 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
   ---------------------------------------- 0.0/42.3 MB ? eta -:--:--
   -- ------------------------------------- 2.4/42.3 MB 11.2 MB/s eta 0:00:04
   ---- ----------------------------------- 4.7/42.3 MB 11.4 MB/s eta 0:00:04
   ----- ---------------------------------- 6.3/42.3 MB 10.4 MB/s eta 0:00:04
   ------- -------------------------------- 8.1/42.3 MB 9.7 MB/s eta 0:00:04
   --------- ------------------------------ 10.5/42.3 MB 10.1 MB/s eta 0:00:04
   ------------ --------------------------- 12.8/42.3 MB 10.1 MB/s eta 0:00:03
   ------------- -------------------------- 14.4/42.3 MB 9.8 MB/s eta 0:00:03
   --------------- ------------------------ 16.3/42.3 MB 9.7 MB/s eta 0:00:03
   ----------------- ---------------------- 18.1/42.3 MB 9.6 MB/s eta 0:00:03
   ------------------ --------------------- 19.9/42.3 MB 9.5 MB/s eta 0:00:03
   -------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-cloud-auth 0.1.4 requires pydantic<2.0, but you have pydantic 2.9.2 which is incompatible.


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://edcbc83ec6c3b618b2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
